## run this cell before starting spark cluster connection (corrects module error)

In [12]:
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [13]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [14]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', None)

### select the data

In [15]:
df_ps = spark.read.json("/user/sprenner/training_official_april_hourly_proportion_prefetching.json")

In [16]:
df_ps.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_ps.columns]).limit(10).toPandas()

account  accountHash  clientState  clientStateHash  count  dataset  \
0        0            0            0                0   1624        0   

   datasetHash  datasetScope  datasetScopeHash  dataset_version  \
0            0             0                 0                0   

   dataset_versionHash  datatype  datatypeHash  eventType  eventTypeHash  \
0                    0         0             0          0              0   

   eventVersion  eventVersionHash  fileScope  filename  filenameHash  \
0             0                 0          0         0             0   

   filesize_double  hits  hostname  hostnameHash  hour  hourHash  id  \
0                0     0         0             0     0         0   0   

   id_proportion  int_percentage  join_key_april  join_key_lookup  \
0         197902          197902               0             1624   

   max_percentage  prod_step  prod_stepHash  project  projectHash  protocol  \
0          111420          0              0        0            0         0   

   protocolHash  remoteSite  remoteSiteHash  run_number  run_numberHash  \
0             0           0               0           0               0   

   scopeHash  stream_name  stream_nameHash  traceId  traceIdHash  traceIp  \
0          0            0                0        0            0        0   

   traceIpHash  traceTimeentryUnix  transferStart  uuid  uuidHash  
0            0                   0              0     0         0

In [17]:
df_ps = df_ps.fillna(0,['count', 'int_percentage','max_percentage'])

In [18]:
df_ps.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_ps.columns]).limit(10).toPandas()

account  accountHash  clientState  clientStateHash  count  dataset  \
0        0            0            0                0      0        0   

   datasetHash  datasetScope  datasetScopeHash  dataset_version  \
0            0             0                 0                0   

   dataset_versionHash  datatype  datatypeHash  eventType  eventTypeHash  \
0                    0         0             0          0              0   

   eventVersion  eventVersionHash  fileScope  filename  filenameHash  \
0             0                 0          0         0             0   

   filesize_double  hits  hostname  hostnameHash  hour  hourHash  id  \
0                0     0         0             0     0         0   0   

   id_proportion  int_percentage  join_key_april  join_key_lookup  \
0         197902               0               0             1624   

   max_percentage  prod_step  prod_stepHash  project  projectHash  protocol  \
0               0          0              0        0            0         0   

   protocolHash  remoteSite  remoteSiteHash  run_number  run_numberHash  \
0             0           0               0           0               0   

   scopeHash  stream_name  stream_nameHash  traceId  traceIdHash  traceIp  \
0          0            0                0        0            0        0   

   traceIpHash  traceTimeentryUnix  transferStart  uuid  uuidHash  
0            0                   0              0     0         0

### make buckets

In [19]:
from pyspark.ml.feature import Bucketizer

splits_percentage = [-float("inf"), 81, float("inf")]
bucketizer = Bucketizer(splits=splits_percentage, inputCol="max_percentage", outputCol="bucketedPercentage")
df_ps = bucketizer.transform(df_ps)

splits_hits = [-float("inf"), 2, float("inf")]
bucketizer = Bucketizer(splits=splits_hits, inputCol="hits", outputCol="bucketedHits")
bucketedData = bucketizer.transform(df_ps)
bucketedData.createOrReplaceTempView("bucket")

In [20]:
spark.sql("SELECT bucketedPercentage, count(*) \
            FROM bucket \
            GROUP BY  bucketedPercentage \
            ORDER BY 1 ASC").toPandas()

#15: 0 = 247128, 1 = 530158
#30: 0 = 350117, 1 = 427169
#50: 0 = 447056, 1 = 330230
#80: 0 = 527559, 1 = 249727

bucketedPercentage  count(1)
0                 0.0    527559
1                 1.0    249727

In [21]:
spark.sql("SELECT bucketedHits, count(*) \
            FROM bucket \
            GROUP BY  bucketedHits \
            ORDER BY 1 ASC").toPandas()

#1: 0 = 541829, 1 = 235457

bucketedHits  count(1)
0           0.0    541829
1           1.0    235457

In [22]:
final_data = bucketedData.withColumn("prefetch_bucket", when(bucketedData["bucketedPercentage"] == 1.0, "prefetch").otherwise("noprefetch"))

In [23]:
final_data = final_data.withColumn("class_bucket", when(final_data["bucketedHits"] == 0.0, "nocache").otherwise("cache"))

In [24]:
final_data.createOrReplaceTempView("final")
spark.sql("SELECT prefetch_bucket, count(*) \
            FROM final \
            GROUP BY  prefetch_bucket \
            ORDER BY 1 ASC").toPandas()

prefetch_bucket  count(1)
0      noprefetch    527559
1        prefetch    249727

In [25]:
spark.sql("SELECT class_bucket, count(*) \
            FROM final \
            GROUP BY  class_bucket \
            ORDER BY 1 ASC").toPandas()

class_bucket  count(1)
0        cache    235457
1      nocache    541829

In [26]:
final_data.describe().toPandas()

summary  account           accountHash         clientState  \
0   count   777286                777286              777286   
1    mean     None   1.935684591732567E8                None   
2  stddev     None  1.1361317241087065E9                None   
3     min  aagaard           -2127257360  CannotAuthenticate   
4     max   zzheng            2143592175      SourceNotFound   

        clientStateHash              count  \
0                777286             777286   
1   -9.74016245191591E8  3673.703860612439   
2  2.6212986830087823E8  7419.355144319622   
3           -1922357674                  0   
4            1269532611             199996   

                                             dataset           datasetHash  \
0                                             777286                777286   
1                                               None  -1.072936835947129E7   
2                                               None  1.2586751291558871E9   
3  data10_hi.00169927.physics_bulk.ROOT.NTUP_HI.r...           -2146605558   
4  mc16_valid.423105.Pythia8EvtGen_A14NNPDF23LO_g...            2147407155   

  datasetScope      datasetScopeHash dataset_version   dataset_versionHash  \
0       777286                777286          777286                777286   
1         None  -7.552649492442358E8            None  1.3991617429934412E7   
2         None   8.314865501449597E8            None  1.3045708543551397E9   
3    data10_hi           -2057006107     c1130_m1847           -2146991447   
4   mc16_valid            2109319881      x557_m1947            2146405934   

  datatype          datatypeHash eventType eventTypeHash eventVersion  \
0   777286                777286    777286        777286       777286   
1     None   4.947060556405943E8      None    5.937808E7         None   
2     None  1.1700303292632794E9      None           0.0         None   
3      AOD           -2070622869  download      59378080       1.13.2   
4      log            2124512252  download      59378080       1.15.5   

       eventVersionHash   fileScope                            filename  \
0                777286      777286                              777286   
1  -1.992468944589716E9        None                                None   
2   3.073273964746096E8        None                                None   
3           -2044360925   data10_hi    AOD.05371635._000001.pool.root.1   
4            1261702107  mc16_valid  log.13787275._000072.job.log.tgz.1   

           filenameHash       filesize_double               hits  \
0                777286                777286             777286   
1   -248557.46481861244    6.11402028756827E8  2.010655022732945   
2  1.2402658149710774E9  1.2408771809945793E9   2.33759009262949   
3           -2147478094                 304.0                  1   
4            2147482617       1.1991446515E10                 40   

                   hostname           hostnameHash           hour  \
0                    777286                 777286         777286   
1                      None  -1.1804844919942595E8           None   
2                      None   1.2286452227051158E9           None   
3    ReCaS-ui-01.cs.infn.it            -2133418459  2018-04-01 00   
4  xenia.nevis.columbia.edu             2129595856  2018-04-30 23   

                hourHash                 id       id_proportion  \
0                 777286             777286              579384   
1  -2.5176970566176414E7           388643.5   370425.3149948911   
2   1.2215909376898775E9  224383.2850061846  228391.37208069928   
3            -2138243719                  1                   2   
4             2144044799             777286              777286   

       int_percentage                                     join_key_april  \
0              777286                                             777286   
1  23.303134238877323                                               None   
2   28.43894253207948                                    

In [27]:
#final_data = final_data.withColumn("fileScopeHash", hash(final_data['fileScope']))

## Bucketing and Hashing End

In [28]:
final_data = final_data.orderBy("traceTimeentryUnix", ascending=True)

In [29]:
final_data.createOrReplaceTempView('final')
final_data = spark.sql("SELECT ROW_NUMBER() OVER (ORDER BY traceTimeentryUnix) as idx, * FROM final")

In [30]:
final_data.createOrReplaceTempView('final')
spark.sql("SELECT * FROM final WHERE idx <= 621828").count()

621828

In [31]:
spark.sql("SELECT * FROM final WHERE idx > 621828").count()

155458

In [32]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

assembler = VectorAssembler(inputCols=["accountHash","clientStateHash","count","datasetHash","datasetScopeHash","dataset_versionHash",
                                       "datatypeHash","eventTypeHash","eventVersionHash","filenameHash", "scopeHash","filesize_double","hostnameHash",
                                       "hourHash","int_percentage","prod_stepHash","projectHash","protocolHash","remoteSiteHash","run_numberHash",
                                       "stream_nameHash","traceIdHash","traceIpHash","traceTimeentryUnix","transferStart","uuidHash"], 
                            outputCol="features")
output = assembler.transform(final_data)
#output = output.select("bucketedFeatures", "features")

output.createOrReplaceTempView('out')
train_data = spark.sql("SELECT * FROM out WHERE idx <= 621828")
test_data = spark.sql("SELECT * FROM out WHERE idx > 621828")

#train_data, test_data = output.randomSplit([.8,.2],seed=1234)
rf_percentage = RandomForestClassifier(labelCol='bucketedPercentage', featuresCol='features', predictionCol="prediction_percentage", numTrees=300, seed=1234)
model_percentage = rf_percentage.fit(train_data)
predictions_percentage = model_percentage.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="bucketedPercentage", predictionCol="prediction_percentage", metricName="accuracy")
accuracy = evaluator.evaluate(predictions_percentage)
print("Test Error Percentage = %g" % (1.0 - accuracy))
predictions_percentage = predictions_percentage.drop('rawPrediction','probability')


rf_hits = RandomForestClassifier(labelCol='bucketedHits', featuresCol='features', predictionCol="prediction_hits",numTrees=300, seed=1234)
model_hits = rf_hits.fit(train_data)
predictions_hits_percentage = model_hits.transform(predictions_percentage)

evaluator = MulticlassClassificationEvaluator(labelCol="bucketedHits", predictionCol="prediction_hits", metricName="accuracy")
accuracy = evaluator.evaluate(predictions_hits_percentage)
print("Test Error Hits = %g" % (1.0 - accuracy))

#Test Error percentage 80: 0.061708
#Test Error Percentage 50 = 0.115999
#Test Error Percentage 30 = 0.141504
#Test Error Percentage 15 = 0.0931055


#Test Error Hits 50 = 0.0643839

Test Error Percentage = 0.061708
Test Error Hits = 0.0643839


In [33]:
model_percentage.featureImportances

SparseVector(26, {0: 0.0326, 1: 0.0192, 2: 0.1255, 3: 0.0032, 4: 0.0213, 5: 0.0029, 6: 0.1878, 8: 0.0, 9: 0.0, 10: 0.0149, 11: 0.1516, 12: 0.0175, 13: 0.0016, 14: 0.3239, 15: 0.0068, 16: 0.0161, 17: 0.0005, 18: 0.0026, 19: 0.0024, 20: 0.0027, 21: 0.0, 22: 0.0117, 23: 0.0318, 24: 0.0192, 25: 0.0043})

In [34]:
model_hits.featureImportances

SparseVector(26, {0: 0.0217, 1: 0.268, 2: 0.0079, 3: 0.0007, 4: 0.0074, 5: 0.0113, 6: 0.0471, 8: 0.0, 9: 0.0, 10: 0.0093, 11: 0.1041, 12: 0.0099, 13: 0.0005, 14: 0.4075, 15: 0.0069, 16: 0.0072, 17: 0.0296, 18: 0.0056, 19: 0.0004, 20: 0.0076, 21: 0.0, 22: 0.0263, 23: 0.008, 24: 0.0074, 25: 0.0053})

In [35]:
predictions_hits_percentage = predictions_hits_percentage.withColumn("filesize", predictions_hits_percentage["filesize_double"].cast(IntegerType()))

In [36]:
final_output = predictions_hits_percentage.select('uuid', 'account', 'datasetScope', 'dataset', 'fileScope','filename', 'eventType', 'clientState', 
                                  from_unixtime(predictions_hits_percentage.traceTimeentryUnix, format='yyyy-MM-dd').alias('day'), 'traceTimeentryUnix', 
                                  'filesize', 'prediction_hits', 'int_percentage','prediction_percentage').sort(col("day").asc())

In [37]:
final_output.count()

155458

In [38]:
final_output.limit(10).toPandas()

uuid   account  datasetScope  \
0  8d160961f13a4effbc0a381a6518d5ca  tofitsch    mc15_14TeV   
1  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
2  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
3  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
4  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
5  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
6  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
7  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
8  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
9  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   

                                             dataset     fileScope  \
0  mc15_14TeV.167796.Sherpa_CT10_WtaunuMassiveCBP...    mc15_14TeV   
1  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
2  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
3  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
4  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
5  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
6  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
7  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
8  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
9  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   

                                    filename eventType         clientState  \
0   DAOD_TRUTH3.13422301._000001.pool.root.1  download                DONE   
1  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
2  DAOD_HIGG5D1.12819918._000581.pool.root.1  download  ServiceUnavailable   
3  DAOD_HIGG5D1.12819918._000581.pool.root.1  download  ServiceUnavailable   
4  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
5  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
6  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
7  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
8  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
9  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   

          day  traceTimeentryUnix    filesize  prediction_hits  \
0  2018-04-22        1.524413e+09   236600961              0.0   
1  2018-04-22        1.524413e+09  1327098000              1.0   
2  2018-04-22        1.524413e+09  1379641362              1.0   
3  2018-04-22        1.524413e+09  1379641362              1.0   
4  2018-04-22        1.524413e+09  1327098000              1.0   
5  2018-04-22        1.524413e+09  1327098000              1.0   
6  2018-04-22        1.524413e+09  1398108348              1.0   
7  2018-04-22        1.524413e+09  1398108348              1.0   
8  2018-04-22        1.524413e+09  1398108348              1.0   
9  2018-04-22        1.524413e+09  1398108348              1.0   

   int_percentage  prediction_percentage  
0             100                    1.0  
1               0                    0.0  
2               0                    0.0  
3               0                    0.0  
4               0                    0.0  
5               0                    0.0  
6               0                    0.0  
7               0                    0.0  
8               0                    0.0  
9               0                    0.0

In [39]:
final_output = final_output.orderBy("traceTimeentryUnix", ascending=True)
final_output.limit(10).toPandas()

uuid   account  datasetScope  \
0  8d160961f13a4effbc0a381a6518d5ca  tofitsch    mc15_14TeV   
1  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
2  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
3  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
4  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
5  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
6  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
7  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
8  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
9  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   

                                             dataset     fileScope  \
0  mc15_14TeV.167796.Sherpa_CT10_WtaunuMassiveCBP...    mc15_14TeV   
1  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
2  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
3  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
4  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
5  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
6  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
7  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
8  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
9  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   

                                    filename eventType         clientState  \
0   DAOD_TRUTH3.13422301._000001.pool.root.1  download                DONE   
1  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
2  DAOD_HIGG5D1.12819918._000581.pool.root.1  download  ServiceUnavailable   
3  DAOD_HIGG5D1.12819918._000581.pool.root.1  download  ServiceUnavailable   
4  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
5  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
6  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
7  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
8  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
9  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   

          day  traceTimeentryUnix    filesize  prediction_hits  \
0  2018-04-22        1.524413e+09   236600961              0.0   
1  2018-04-22        1.524413e+09  1327098000              1.0   
2  2018-04-22        1.524413e+09  1379641362              1.0   
3  2018-04-22        1.524413e+09  1379641362              1.0   
4  2018-04-22        1.524413e+09  1327098000              1.0   
5  2018-04-22        1.524413e+09  1327098000              1.0   
6  2018-04-22        1.524413e+09  1398108348              1.0   
7  2018-04-22        1.524413e+09  1398108348              1.0   
8  2018-04-22        1.524413e+09  1398108348              1.0   
9  2018-04-22        1.524413e+09  1398108348              1.0   

   int_percentage  prediction_percentage  
0             100                    1.0  
1               0                    0.0  
2               0                    0.0  
3               0                    0.0  
4               0                    0.0  
5               0                    0.0  
6               0                    0.0  
7               0                    0.0  
8               0                    0.0  
9               0                    0.0

## get complete amount of data downloaded in time window to calculate good cache size

In [40]:
final_output.createOrReplaceTempView('output')
spark.sql("SELECT SUM(filesize) FROM (SELECT DISTINCT fileScope, fileName, filesize FROM output)").toPandas()
#36398774222862

sum(filesize)
0  36398774222862

In [41]:
#final_output.write.csv('/user/sprenner/forest_april_official_hourly_ordered_prefetching15_numTrees300_seed1234_cache01.csv', sep='\t')

## predict downloads for may with trained model

In [42]:
df_mai = spark.read.json("/user/sprenner/training_official_mai_hourly_proportion_prefetching.json")

In [43]:
df_mai.describe().toPandas()

summary   account           accountHash         clientState  \
0   count    589283                589283              589283   
1    mean      None  -7.873822054121874E7                None   
2  stddev      None  1.1303640225554883E9                None   
3     min     abell           -2123889579  CannotAuthenticate   
4     max  zmarshal            2140382084      SourceNotFound   

         clientStateHash               count  \
0                 589283              578132   
1  -1.0635692705599144E9  1573.6514117883114   
2    2.754569203849525E8  10628.459825186354   
3            -1913538301                   1   
4              634051254              195454   

                                             dataset           datasetHash  \
0                                             589283                589283   
1                                               None  -3.916131021861822E7   
2                                               None  1.2336014929229865E9   
3  data12_8TeV.00200805.physics_MinBias.recon.NTU...           -2147225360   
4  mc16_valid.361034.Pythia8EvtGen_A2MSTW2008LO_m...            2147407155   

  datasetScope      datasetScopeHash dataset_version   dataset_versionHash  \
0       589283                589283          589283                589283   
1         None  -9.356107326362851E8            None  -7.824071159882942E7   
2         None   6.784957656574697E8            None  1.1845690301436512E9   
3  data12_8TeV           -2057006107     c1128_m1842           -2147305504   
4   mc16_valid            2109319881      x563_m1972            2147255052   

  datatype          datatypeHash eventType eventTypeHash eventVersion  \
0   589283                589283    589283        589283       589283   
1     None  2.1895068833955333E8      None    5.937808E7         None   
2     None  1.0743326832603405E9      None           0.0         None   
3      AOD           -2070622869  download      59378080      1.14.10   
4      log            2124512252  download      59378080       1.16.1   

        eventVersionHash    fileScope                            filename  \
0                 589283       589283                              589283   
1  -2.0115120046498694E9         None                                None   
2    3.165646873326364E8         None                                None   
3            -2044360925  data12_8TeV    AOD.05341490._000446.pool.root.1   
4             1261702107   mc16_valid  log.14217468._003414.job.log.tgz.1   

           filenameHash       filesize_double                hits  \
0                589283                589283              589283   
1     684033.7623824207  1.4339739956710258E9  2.9973221694839323   
2  1.2390276453463626E9   2.193139944581198E9  3.8806741048619773   
3           -2147471303                9576.0                   1   
4            2147478625       1.1976018886E10                  74   

                   hostname          hostnameHash           hour  \
0                    589283                589283         589283   
1                      None  -4.071422388835245E7           None   
2                      None   9.756384901185813E8           None   
3    ReCaS-ui-01.cs.infn.it           -2138306752  2018-05-01 00   
4  xenia.nevis.columbia.edu            2129595856  2018-05-31 23   

               hourHash                  id       id_proportion  \
0                589283              589283              357853   
1    -3414275.108615046            294642.0     300680.44414606   
2  1.2200577022275505E9  170111.49367694117  163494.92291812465   
3           -2140578122                   1                   2   
4            2146843273              589283              589283   

       int_percentage                                    join_key_lookup  \
0              357853                                             578132   
1  31.659882689260673                                               None   
2  31.138705245748017      

In [44]:
df_mai = df_mai.fillna(0,['count', 'int_percentage','max_percentage'])

In [45]:
splits_percentage_mai = [-float("inf"), 16, float("inf")]
bucketizer_mai = Bucketizer(splits=splits_percentage_mai, inputCol="max_percentage", outputCol="bucketedPercentage")
df_mai = bucketizer_mai.transform(df_mai)

splits_hits_mai = [-float("inf"), 2, float("inf")]
bucketizer_mai = Bucketizer(splits=splits_hits_mai, inputCol="hits", outputCol="bucketedHits")
bucketedData_mai = bucketizer.transform(df_mai)
bucketedData_mai.createOrReplaceTempView("bucket_mai")

#df_mai_final = bucketedData_mai.withColumn("fileScopeHash", hash(bucketedData_mai['fileScope']))
df_mai_final = bucketedData_mai.orderBy("traceTimeentryUnix", ascending=True)

In [46]:
assembler_mai = VectorAssembler(
    inputCols=["accountHash","clientStateHash","count","datasetHash","datasetScopeHash","dataset_versionHash",
                "datatypeHash","eventTypeHash","eventVersionHash","filenameHash", "scopeHash","filesize_double","hostnameHash",
                "hourHash","int_percentage","prod_stepHash","projectHash","protocolHash","remoteSiteHash","run_numberHash",
                "stream_nameHash","traceIdHash","traceIpHash","traceTimeentryUnix","transferStart","uuidHash"],
    outputCol="features")

output_mai = assembler_mai.transform(df_mai_final)

In [47]:
predictions_percentage_mai = model_percentage.transform(output_mai)
evaluator = MulticlassClassificationEvaluator(labelCol="bucketedPercentage", predictionCol="prediction_percentage", metricName="accuracy")
accuracy = evaluator.evaluate(predictions_percentage_mai)
print("Test Error Percentage Mai = %g" % (1.0 - accuracy))
predictions_percentage_mai = predictions_percentage_mai.drop('rawPrediction','probability')

predictions_hits_percentage_mai = model_hits.transform(predictions_percentage_mai)

evaluator = MulticlassClassificationEvaluator(labelCol="bucketedHits", predictionCol="prediction_hits", metricName="accuracy")
accuracy = evaluator.evaluate(predictions_hits_percentage_mai)
print("Test Error Hits Mai = %g" % (1.0 - accuracy))
#percentage mai 80: 0.123328
#percentage mai 50: 0.119174
#percentage mai 30: 0.117972
#percentage mai 15: 0.165669

#hits mai 80: 0.127231

Test Error Percentage Mai = 0.332095
Test Error Hits Mai = 0.127231


In [48]:
predictions_hits_percentage_mai=predictions_hits_percentage_mai.withColumn("filesize", predictions_hits_percentage_mai["filesize_double"].cast(IntegerType()))

In [49]:
predictions_hits_percentage_mai = predictions_hits_percentage_mai.drop('filesize_double')

In [50]:
final_output_mai = predictions_hits_percentage_mai.select('uuid', 'account', 'datasetScope', 'dataset', 'fileScope','filename', 'eventType', 'clientState', 
                                  from_unixtime(predictions_hits_percentage_mai.traceTimeentryUnix, format='yyyy-MM-dd').alias('day'), 'traceTimeentryUnix', 
                                  'filesize', 'prediction_hits', 'int_percentage','prediction_percentage').sort(col("day").asc())

In [51]:
final_output_mai = final_output_mai.orderBy("traceTimeentryUnix", ascending=True)

In [52]:
final_output_mai.createOrReplaceTempView('output_mai')
spark.sql("SELECT SUM(filesize) FROM (SELECT DISTINCT fileScope, fileName, filesize FROM output_mai)").toPandas()
#209595321347926

sum(filesize)
0  209595321347926

In [53]:
final_output_mai.count()

589283

In [54]:
final_output_mai = final_output_mai.orderBy("traceTimeentryUnix", ascending=True)

In [55]:
#final_output_mai.write.csv('/user/sprenner/forest_mai_official_hourly_ordered_prefetching15_numTrees300_seed1234_cache01.csv', sep='\t')